In [14]:
!pip install librosa

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
# Path to save outputs
root_path = '/content/drive/MyDrive/AIMusicGenerator/music_voice/'
os.makedirs(root_path, exist_ok=True)
output_midi_mp3 = os.path.join(root_path,"melody.mid")
output_audio_mp3 = os.path.join(root_path,"singing.mp3")
output_audio_wav = os.path.join(root_path,"singing.wav")
output_musicxml = os.path.join(root_path, "twinkle_twinkle.musicxml")

In [ ]:
from pydub import AudioSegment
from midiutil import MIDIFile
from gtts import gTTS
import simpleaudio as sa
import os

# Step 1: Generate a melody as a MIDI file
def create_melody(notes, durations, output_file="melody.mid", tempo=120):
    """
    Create a melody as a MIDI file.

    Parameters:
    - notes: List of MIDI note numbers.
    - durations: List of durations for each note (in beats).
    - output_file: Name of the output MIDI file.
    - tempo: Tempo of the melody in beats per minute.
    """
    midi = MIDIFile(1)
    midi.addTempo(0, 0, tempo)
    time = 0

    for note, duration in zip(notes, durations):
        midi.addNote(0, 0, note, time, duration, 100)
        time += duration

    with open(output_file, "wb") as f:
        midi.writeFile(f)

    print(f"Melody saved as {output_file}")


# Example melody: "Twinkle Twinkle Little Star"
notes = [60, 60, 67, 67, 69, 69, 67, 65, 65, 64, 64, 62, 62, 60]  # MIDI notes
durations = [1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2]  # Beats per note
create_melody(notes, durations, output_file=output_midi_mp3)

# Step 2: Generate lyrics and convert to audio
lyrics = "Twinkle twinkle little star"


def synthesize_lyrics(lyrics, output_mp3, output_wav):
    """
    Synthesize lyrics into audio using gTTS and convert to .wav format.

    Parameters:
    - lyrics: Text of the lyrics to synthesize.
    - output_mp3: Name of the output MP3 file.
    - output_wav: Name of the output WAV file.
    """
    # Generate MP3
    tts = gTTS(lyrics, lang="en")
    tts.save(output_mp3)

    # Convert MP3 to WAV
    audio = AudioSegment.from_file(output_mp3, format="mp3")
    audio.export(output_wav, format="wav")
    print(f"Lyrics saved as audio in {output_wav}")

synthesize_lyrics(lyrics, output_audio_mp3, output_audio_wav)

# Step 3: Combine melody and lyrics (playback)
def play_audio(audio_file):
    """
    Play an audio file using simpleaudio.

    Parameters:
    - audio_file: Path to the audio file.
    """
    wave_obj = sa.WaveObject.from_wave_file(audio_file)
    play_obj = wave_obj.play()
    play_obj.wait_done()

# print("Playing synthesized singing voice...")
# play_audio(output_audio_wav)

# Download the song into local computer


Melody saved as /content/drive/MyDrive/AIMusicGenerator/music_voice/melody.mid
Lyrics saved as audio in /content/drive/MyDrive/AIMusicGenerator/music_voice/singing.wav


In [13]:
from music21 import stream, note, metadata

def create_musicxml_with_lyrics(notes, durations, lyrics, output_file):
    """
    Create a MusicXML file with melody and lyrics.

    Parameters:
    - notes: List of MIDI note numbers.
    - durations: List of durations for each note (in quarter lengths).
    - lyrics: List of lyrics corresponding to the notes.
    - output_file: Name of the output MusicXML file.
    """
    s = stream.Part()
    s.metadata = metadata.Metadata()
    s.metadata.title = "Twinkle Twinkle Little Star"
    s.metadata.composer = "AI Generator"

    for pitch, dur, lyric in zip(notes, durations, lyrics):
        n = note.Note(pitch)
        n.quarterLength = dur
        if lyric:
            n.lyric = lyric
        s.append(n)

    # Write to MusicXML
    s.write('musicxml', fp=output_file)
    print(f"MusicXML saved as {output_file}")

# Example input
lyrics_list = ["Twinkle", "twinkle", "little", "star", "", "", "how", "I", "wonder", "what", "you", "are", "", ""]
create_musicxml_with_lyrics(notes, durations, lyrics_list, output_musicxml)


MusicXML saved as /content/drive/MyDrive/AIMusicGenerator/music_voice/twinkle_twinkle.musicxml
